In [1]:
import dgl
import dgl.function as fn

import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


In [2]:
import pickle

# Load the training and validation molecules.
train_path = "data/zinc_250k_train.pkl"
valid_path = "data/zinc_250k_valid.pkl"

with open(train_path, "rb") as f:
    train_graphs = pickle.load(f)

with open(valid_path, "rb") as f:
    valid_graphs = pickle.load(f)

In [3]:
from utils import to_dgl_graph

# Turn the molecules into DGLGraph, so as to train a model with the Deep Graph Library (DGL).
train_dataset = []
valid_dataset = []

for graph in train_graphs:
    train_dataset.append(to_dgl_graph(graph))

for graph in valid_graphs:
    valid_dataset.append(to_dgl_graph(graph))

nb_node_types = 14
nb_edge_types = 3

RDKit WARNING: [18:24:39] Enabling RDKit 2019.09.3 jupyter extensions


In [4]:
from torch.utils.data import DataLoader

# Create mini-batches
def collate(samples):
    return dgl.batch(samples)

train_loader = DataLoader(train_dataset, batch_size=32, collate_fn=collate, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, collate_fn=collate)

In [7]:
from model import VGAE

# Create the model and the optimizer
device = "cpu"
n_epochs = 2
lr = 0.01

model = VGAE(nb_node_types, nb_edge_types).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

print(model)

VGAE(
  (conv1): RelGraphConv(
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (conv_mu): RelGraphConv(
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (conv_logstd): RelGraphConv(
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (node_types_fc): Linear(in_features=32, out_features=14, bias=True)
  (edge_existence_fc): Linear(in_features=124, out_features=1, bias=True)
  (edge_types_fc): Linear(in_features=124, out_features=3, bias=True)
)


In [8]:
from tqdm import tqdm
from utils import recon_loss, kl_loss

def eval(dataloader, model, device):
    eval_loss = 0
    model.eval()

    with torch.no_grad():
        for bg in dataloader:
            bg = bg.to(device)

            n_features = bg.ndata["feats"]
            e_types = bg.edata["type"]

            pred_graphs, mu, logstd = model(bg, n_features, e_types)
            loss = recon_loss(pred_graphs, dgl.unbatch(bg), device) + kl_loss(mu, logstd)
            eval_loss += loss.item()

    model.train()
    return eval_loss / len(dataloader)

# Train the model 
for epoch in range(n_epochs):
    train_loss = 0
    model.train()

    for bg in tqdm(train_loader):
        bg = bg.to(device)
            
        n_features = bg.ndata["feats"]
        e_types = bg.edata["type"]

        pred_graphs, mu, logstd = model(bg, n_features, e_types)

        loss = recon_loss(pred_graphs, dgl.unbatch(bg), device) + kl_loss(mu, logstd)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)
    valid_loss = eval(valid_loader, model, device)
        
    print("Epoch {}, train_loss: {:,.3f}, valid_loss: {:,.3f}".format(epoch, train_loss, valid_loss))

100%|██████████| 7000/7000 [25:54<00:00,  4.50it/s]
Epoch 1, train_loss: 2.210, valid_loss: 2.211


In [9]:
# Save model to disk
torch.save(model.state_dict(), "trained_model.pkl")